In [1]:
import pandas as pd
import numpy as np
import keras
import time

In [2]:
#importing the datasets (train_set_augmented for all landmarks, train_set_augmented_simplified for simplified landmarks)
train_set_full = pd.read_csv('outputs/train_set_augmented.csv')
test_set_left = pd.read_csv('outputs/test_set_augmented_left.csv')
test_set_simplified = pd.read_csv('outputs/test_set_augmented_simplified.csv')

In [3]:
sample_full_df = train_set_full.sample(n=100)
sample_left_df = test_set_left.sample(n=100)
sample_simplified_df = test_set_simplified.sample(n=100)

In [4]:
sample_full_df

,label,lm_1_x,lm_1_y,lm_1_z,lm_2_x,lm_2_y,lm_2_z,lm_3_x,lm_3_y,lm_3_z,...,lm_475_z,lm_476_x,lm_476_y,lm_476_z,lm_477_x,lm_477_y,lm_477_z,lm_478_x,lm_478_y,lm_478_z
24984,happy,0.652054,0.661385,-0.062562,0.632844,0.579676,-0.192819,0.632633,0.602704,-0.091508,...,0.001678,0.704595,0.261006,0.001681,0.684277,0.300500,0.001643,0.725480,0.314542,0.001648
25895,anger,0.538851,0.697791,-0.065943,0.514334,0.613901,-0.149693,0.525563,0.636113,-0.073803,...,-0.025169,0.651204,0.372005,-0.025167,0.625226,0.403877,-0.025200,0.660584,0.424764,-0.025197
20545,happy,0.497425,0.723701,-0.057783,0.513479,0.640790,-0.127577,0.505841,0.661057,-0.066903,...,0.037033,0.612976,0.445067,0.037036,0.590334,0.466863,0.037004,0.614501,0.487313,0.037011
8855,sad,0.491745,0.704081,-0.069988,0.488319,0.612074,-0.142212,0.492104,0.640057,-0.070512,...,0.000039,0.646641,0.406317,0.000040,0.618555,0.428194,0.000008,0.642665,0.454878,0.000013
14092,happy,0.539225,0.691100,-0.085724,0.515884,0.609284,-0.146541,0.524084,0.634652,-0.085328,...,0.021820,0.613164,0.415682,0.021818,0.592292,0.440990,0.021792,0.621581,0.455108,0.021797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17929,sad,0.490755,0.728540,-0.046760,0.469056,0.642964,-0.140551,0.484983,0.659533,-0.059691,...,-0.050693,0.650830,0.382388,-0.050695,0.620489,0.410720,-0.050727,0.653104,0.437649,-0.050723
15461,neutral,0.522712,0.720520,-0.065123,0.540219,0.627364,-0.123877,0.527059,0.651742,-0.064549,...,0.048664,0.615499,0.443881,0.048668,0.592645,0.462174,0.048638,0.613095,0.482613,0.048643
20648,happy,0.512631,0.717056,-0.029503,0.534085,0.669963,-0.121950,0.520558,0.679482,-0.050634,...,0.006491,0.617259,0.435181,0.006493,0.592110,0.458905,0.006461,0.617603,0.482338,0.006468
16250,neutral,0.456350,0.678932,-0.052540,0.460881,0.610934,-0.115702,0.463116,0.625896,-0.054121,...,-0.014871,0.626619,0.464953,-0.014868,0.594184,0.487583,-0.014893,0.618964,0.518484,-0.014888


In [5]:
sample_full_df.iloc[0, 1:]

lm_1_x      0.652054
lm_1_y      0.661385
lm_1_z     -0.062562
lm_2_x      0.632844
lm_2_y      0.579676
              ...   
lm_477_y      0.3005
lm_477_z    0.001643
lm_478_x     0.72548
lm_478_y    0.314542
lm_478_z    0.001648
Name: 24984, Length: 1434, dtype: object

## CNN Performance Testing Results

In [6]:
#importing the datasets (train_set_augmented for all landmarks, train_set_augmented_simplified for simplified landmarks)
stat_full = pd.read_csv('assets/stat_train_features.csv')
stat_left = pd.read_csv('assets/stat_train_features_left.csv')
stat_simplified = pd.read_csv('assets/stat_train_features_simplified.csv')

In [7]:
def scale_face_landmarks(face_landmarks, mean_list, std_list):
    for i in range(len(face_landmarks)):
        face_landmarks[i] = (face_landmarks[i] - mean_list[i]) / std_list[i]

    return face_landmarks

def testModel(emotion_model, sample, stat_df, time_array):
    mean_list = stat_df['mean'].to_list()
    std_list = stat_df['std'].to_list()

    #Go through each image
    for idx, row in sample.iterrows():
        #1. Retrieve the landmarks first
        face_landmarks = row[1:].to_list()
        
        start_time = time.perf_counter()
        face_landmarks = scale_face_landmarks(face_landmarks, mean_list, std_list)
        predicted_emotion = emotion_model.predict(np.array([face_landmarks,]))
        end_time = time.perf_counter()

        elapsed_time = end_time  - start_time
        time_array.append(elapsed_time)
        print(str(elapsed_time), end=",")

### Full Face

In [8]:
emotion_detection_wd = 'assets/emotion_model.keras'

In [9]:
cnn_full_time_array = []
cnn_full_model = keras.models.load_model(emotion_detection_wd)
testModel(cnn_full_model, sample_full_df, stat_full, cnn_full_time_array)



1/1 [==============================] - 0s 13ms/step
0.04320870000083232,

In [10]:
cnn_full_time_array

[0.17233310000301572,
 0.04368409999733558,
 0.04320570000709267,
 0.04357660000096075,
 0.04456580000260146,
 0.043256700002530124,
 0.04378089999954682,
 0.04344719999789959,
 0.043126199998368975,
 0.043697100001736544,
 0.04539889999432489,
 0.04293399999733083,
 0.04325109999626875,
 0.04332079999585403,
 0.043174400001589675,
 0.04357470000104513,
 0.044717600001604296,
 0.04296310000063386,
 0.04361280000011902,
 0.04442099999869242,
 0.04324520000227494,
 0.04331429999729153,
 0.04494010000053095,
 0.04321930000151042,
 0.04317750000336673,
 0.04341190000559436,
 0.043535299999348354,
 0.04343510000035167,
 0.04637499999807915,
 0.04314889999659499,
 0.04309759999887319,
 0.04286129999672994,
 0.04281559999799356,
 0.042937400001392234,
 0.04585930000030203,
 0.043972700004815124,
 0.04298910000215983,
 0.042908100003842264,
 0.0433234000010998,
 0.04326579999906244,
 0.044787600003473926,
 0.04304730000148993,
 0.0429463999971631,
 0.04282539999985602,
 0.04274720000103116,
 0

In [11]:
total_time = 0
for t in cnn_full_time_array:
    total_time += t
total_time /= 100

print("Average Time for CNN Full Model for 100 trials:", total_time * 1000, "milliseconds")

Average Time for CNN Full Model for 100 trials: 44.93892099999357 milliseconds


### Left Face

In [12]:
emotion_detection_wd_left = 'assets/emotion_model_left.keras'

In [13]:
import time
cnn_left_time_array = []
cnn_left_model = keras.models.load_model(emotion_detection_wd_left)
testModel(cnn_left_model, sample_left_df, stat_left, cnn_left_time_array)

1/1 [==============================] - 0s 14ms/step
0.043857099997694604,

In [14]:
cnn_left_time_array

[0.14539479999803007,
 0.04451410000183387,
 0.045345399994403124,
 0.04501719999825582,
 0.04461550000269199,
 0.044229399994947016,
 0.04412649999721907,
 0.04431520000071032,
 0.04594939999515191,
 0.044834100001025945,
 0.044556299995747395,
 0.04415119999612216,
 0.04407950000313576,
 0.044279800000367686,
 0.046922000001359265,
 0.044580000001587905,
 0.04507279999961611,
 0.044072499993490055,
 0.04389380000066012,
 0.04425690000061877,
 0.04547820000152569,
 0.044178999996802304,
 0.044188599997141864,
 0.04370760000165319,
 0.044280000001890585,
 0.04434159999800613,
 0.046075500002189074,
 0.04391500000201631,
 0.04382469999836758,
 0.04351619999943068,
 0.04402500000287546,
 0.04339649999747053,
 0.045441399997798726,
 0.04401859999779845,
 0.043680499999027234,
 0.044107900001108646,
 0.04422590000467608,
 0.04420060000120429,
 0.04556679999950575,
 0.044188100000610575,
 0.043447200005175546,
 0.044524500000989065,
 0.04411370000161696,
 0.044297599997662473,
 0.0458778999

In [15]:
total_time = 0
for t in cnn_left_time_array:
    total_time += t
total_time /= 100

print("Average Time CNN Left Model for 100 trials:", total_time * 1000, "milliseconds")

Average Time CNN Left Model for 100 trials: 45.40784699973301 milliseconds


### Simplified Face

In [16]:
emotion_detection_wd_simplified = 'assets/emotion_model_simplified.keras'

In [17]:
import time
cnn_simplified_time_array = []
cnn_simplified_model = keras.models.load_model(emotion_detection_wd_simplified)
testModel(cnn_simplified_model, sample_simplified_df, stat_simplified, cnn_simplified_time_array)

1/1 [==============================] - 0s 13ms/step
0.04436100000020815,

In [18]:
cnn_simplified_time_array

[0.14512679999461398,
 0.04424060000019381,
 0.04358069999580039,
 0.043740100001741666,
 0.04349530000035884,
 0.04484249999950407,
 0.043101200004457496,
 0.0438477000061539,
 0.043469700001878664,
 0.04315689999930328,
 0.043336000002454966,
 0.04592699999921024,
 0.04397919999610167,
 0.043561200000112876,
 0.0437244000058854,
 0.044679499995254446,
 0.04399540000304114,
 0.045047399995382875,
 0.04288010000163922,
 0.043411500002548564,
 0.04395449999719858,
 0.04418889999942621,
 0.04372929999954067,
 0.045095200002833735,
 0.04334530000051018,
 0.04304060000140453,
 0.04299330000503687,
 0.04351209999731509,
 0.043900799995753914,
 0.0456382999982452,
 0.043754999998782296,
 0.04433189999690512,
 0.043675899993104395,
 0.043259599995508324,
 0.04343210000661202,
 0.04502749999664957,
 0.043281299993395805,
 0.04355590000341181,
 0.0435693999970681,
 0.04393440000421833,
 0.043689699996320996,
 0.045001300000876654,
 0.043329200001608115,
 0.046280800001113676,
 0.045808400005626

In [19]:
total_time = 0
for t in cnn_simplified_time_array:
    total_time += t
total_time /= 100

print("Average Time CNN Simplified Model for 100 trials:", total_time * 1000, "milliseconds")

Average Time CNN Simplified Model for 100 trials: 44.911859999920125 milliseconds
